In [ ]:
import os
os.chdir("/Users/urijzuzaev/Desktop/Thesis/reviews_dataset/short_dataset.csv")
import pandas as pd
import re
!pip install deep-translator
from deep_translator import GoogleTranslator
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
sid = SentimentIntensityAnalyzer()
!pip install pymorphy2
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
import string
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('russian')

In [ ]:
df = pd.read_csv('/short_dataset.csv')

In [ ]:
results = []
c = 0
for sentence, sentiment in zip(df['Review'], df['Sentiment']):
    c += 1
    print(c)
    new_sentence = sentence[:2000] # since the translator cannot handle sentences longer than 2000 symbols, we are shortening those sentences that are beyond the limit
    translated = GoogleTranslator(source='auto', target='en').translate(new_sentence)
    sid = SentimentIntensityAnalyzer()
    sentiment_scores = sid.polarity_scores(translated)
    pos_score = sentiment_scores['pos']
    neg_score = sentiment_scores['neg']
    subtraction = pos_score - neg_score
    if subtraction > 0.1: # if the subtraction between pos and neg scores are more than 0.1, we assign the "positive" label to the review
                          # we do this so as to avoid ambiguous cases when the values are nearly the same
        result = 'positive'
    else:
        result = 'negative'
    results.append(result)

In [ ]:
df['Prediction'] = results

In [ ]:
print(df['Prediction'].value_counts())

In [ ]:
def conf_matrix(sentiment, prediction):
  if sentiment == 'positive' and prediction == 'positive':
    return 'TP'
  elif sentiment == 'positive' and prediction == 'negative':
    return 'FN'
  elif sentiment == 'negative' and prediction == 'positive':
    return 'FP'
  elif sentiment == 'negative' and prediction == 'negative':
    return 'TN'
    
df['Conf_Matrix'] = [conf_matrix(x, y) for x, y in zip(df['Sentiment'], df['Prediction'])]

In [ ]:
conf_vals = test.Conf_Matrix.value_counts().to_dict()
print(conf_vals)

accuracy = (conf_vals['TP'] + conf_vals['TN']) / (conf_vals['TP'] + conf_vals['TN'] + conf_vals['FP'] + conf_vals['FN'])
precision = conf_vals['TP'] / (conf_vals['TP'] + conf_vals['FP'])
recall = conf_vals['TP'] / (conf_vals['TP'] + conf_vals['FN'])
f1_score = 2*precision*recall / (precision + recall)
print('Accuracy: ', round(100 * accuracy, 2),'%',
      '\nPrecision: ', round(100 * precision, 2),'%',
      '\nRecall: ', round(100 * recall, 2),'%',
      '\nF1 Score: ', round(100 * f1_score, 2),'%')